In [1]:
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
ticker = "BBAS3.SA"
start_date = "2010-01-01"
end_date = "2024-11-23"  # Atualize para a data atual

# Baixar os dados históricos
data = yf.download(ticker, start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [3]:
dividends = yf.Ticker(ticker).dividends
data['Dividendos'] = dividends.reindex(data.index).fillna(0)
#Possui os dados diários apenas de: preço de abertura, fechamento, maior  e menor preço do dia, e quantas ações negociadas
data

Price,Adj Close,Close,High,Low,Open,Volume,Dividendos
Ticker,BBAS3.SA,BBAS3.SA,BBAS3.SA,BBAS3.SA,BBAS3.SA,BBAS3.SA,
Date,,,,,,,
2010-01-04 00:00:00+00:00,5.401157,14.950000,15.000000,14.850000,14.900000,7249400,0.0
2010-01-05 00:00:00+00:00,5.346966,14.800000,15.050000,14.595000,15.050000,6129800,0.0
2010-01-06 00:00:00+00:00,5.354194,14.820000,15.000000,14.710000,14.745000,5466200,0.0
2010-01-07 00:00:00+00:00,5.355997,14.825000,14.855000,14.705000,14.720000,3084400,0.0
2010-01-08 00:00:00+00:00,5.386707,14.910000,14.910000,14.735000,14.850000,4077800,0.0
...,...,...,...,...,...,...,...
2024-11-14 00:00:00+00:00,24.893829,25.370001,25.790001,24.790001,25.780001,80824400,0.0
2024-11-18 00:00:00+00:00,25.227446,25.709999,25.809999,25.340000,25.379999,23386200,0.0


In [4]:
def calculate_rsi(data, window=14):
    # Calcula a diferença diária nos preços ajustados
    delta = data['Adj Close'].diff()

    # Separa os ganhos e perdas
    gain = delta.clip(lower=0)  # Apenas valores positivos
    loss = -delta.clip(upper=0)  # Apenas valores negativos

    # Cálculo inicial para os primeiros valores (primeiros `window` dias)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()

    # Evita divisão por zero (se `avg_loss` for 0, define RSI como 100)
    rs = avg_gain / avg_loss
    rs = rs.replace([float('inf'), float('-inf')], 0)  # Substituir infinitos por 0

    rsi = 100 - (100 / (1 + rs))
    rsi[:window] = None  # Define os primeiros valores como NaN, já que não são confiáveis

    return rsi

def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
    ema_short = data['Adj Close'].ewm(span=short_window, adjust=False).mean()
    ema_long = data['Adj Close'].ewm(span=long_window, adjust=False).mean()
    macd = ema_short - ema_long
    signal = macd.ewm(span=signal_window, adjust=False).mean()
    return macd, signal

data['RSI'] = calculate_rsi(data)


data['MACD'], data['Signal Line'] = calculate_macd(data)


In [5]:
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3686 entries, 2010-01-22 00:00:00+00:00 to 2024-11-22 00:00:00+00:00
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   (Adj Close, BBAS3.SA)  3686 non-null   float64
 1   (Close, BBAS3.SA)      3686 non-null   float64
 2   (High, BBAS3.SA)       3686 non-null   float64
 3   (Low, BBAS3.SA)        3686 non-null   float64
 4   (Open, BBAS3.SA)       3686 non-null   float64
 5   (Volume, BBAS3.SA)     3686 non-null   int64  
 6   (Dividendos, )         3686 non-null   float64
 7   (RSI, )                3686 non-null   float64
 8   (MACD, )               3686 non-null   float64
 9   (Signal Line, )        3686 non-null   float64
dtypes: float64(9), int64(1)
memory usage: 316.8 KB


In [6]:
data

Price,Adj Close,Close,High,Low,Open,Volume,Dividendos,RSI,MACD,Signal Line
Ticker,BBAS3.SA,BBAS3.SA,BBAS3.SA,BBAS3.SA,BBAS3.SA,BBAS3.SA,,,,
Date,,,,,,,,,,
2010-01-22 00:00:00+00:00,5.155488,14.270000,14.350000,13.970000,13.970000,5010400,0.0,34.259464,-0.049611,-0.023700
2010-01-26 00:00:00+00:00,5.016392,13.885000,14.100000,13.755000,14.000000,7566000,0.0,30.897794,-0.067388,-0.032438
2010-01-27 00:00:00+00:00,4.976650,13.775000,14.000000,13.600000,13.855000,6057000,0.0,28.973732,-0.083719,-0.042694
2010-01-28 00:00:00+00:00,4.985683,13.800000,13.995000,13.565000,13.950000,9772200,0.0,29.541092,-0.094839,-0.053123
2010-01-29 00:00:00+00:00,5.076000,14.050000,14.230000,13.835000,13.855000,7035400,0.0,33.894983,-0.095266,-0.061552
...,...,...,...,...,...,...,...,...,...,...
2024-11-14 00:00:00+00:00,24.893829,25.370001,25.790001,24.790001,25.780001,80824400,0.0,23.118212,-0.284973,-0.255183
2024-11-18 00:00:00+00:00,25.227446,25.709999,25.809999,25.340000,25.379999,23386200,0.0,35.813854,-0.288546,-0.261856


In [8]:

data_anualbb = [
    {
       'Ano': 2011 ,  
       'P/L': 5.35,
       'PSR': 0.73,
       'P/VP': 1.17,
       'DY (%)': 7.93,
       'MARGEM LÍQUIDA (%)': 13.61,
       'ROE (%)': 19.91,
       'MARGEM BRUTA (%)': 32.48,
       'MARGEM EBIT (%)': 19.21,
       'P/EBIT': 3.79,
       'ROA (%)': 1.31,
    },
    {
        'Ano': 2012,
        'P/L': 6.52,
        'PSR': 0.78,
        'P/VP': 1.20,
        'DY (%)': 6.33,
        'MARGEM LÍQUIDA (%)': 11.94,
        'ROE (%)': 16.26,
        'MARGEM BRUTA (%)': 32.05,
        'MARGEM EBIT (%)': 15.94,
        'P/EBIT': 4.89,
        'ROA (%)': 0.99,
    },
    {
        'Ano': 2013,
        'P/L': 6.70,
        'PSR': 0.67,
        'P/VP': 1.00,
        'DY (%)': 9.68,
        'MARGEM LÍQUIDA (%)': 9.98,
        'ROE (%)': 14.49,
        'MARGEM BRUTA (%)': 28.88,
        'MARGEM EBIT (%)': 12.30,
        'P/EBIT': 5.44,
        'ROA (%)': 0.90,
    },
    {
        'Ano': 2014,
        'P/L': 5.10,
        'PSR': 0.49,
        'P/VP': 0.90,
        'DY (%)': 6.97,
        'MARGEM LÍQUIDA (%)': 8.60,
        'ROE (%)': 14.49,
        'MARGEM BRUTA (%)': 23.13,
        'MARGEM EBIT (%)': 11.33,
        'P/EBIT': 4.36,
        'ROA (%)': 0.93,
    },
    {
        'Ano': 2015,
        'P/L': 2.67,
        'PSR': 0.23,
        'P/VP': 0.61,
        'DY (%)': 11.42,
        'MARGEM LÍQUIDA (%)': 7.71,
        'ROE (%)': 17.04,
        'MARGEM BRUTA (%)': 12.32,
        'MARGEM EBIT (%)': 5.56,
        'P/EBIT': 4.17,
        'ROA (%)': 1.01,
    },
    {
        'Ano': 2016,
        'P/L': 11.45,
        'PSR': 0.46,
        'P/VP': 1.06,
        'DY (%)': 2.65,
        'MARGEM LÍQUIDA (%)': 3.99,
        'MARGEM EBIT (%)': 6.18,
        'ROE (%)': 8.14,
        'MARGEM BRUTA (%)': 17.58,
        'P/EBIT': 7.39,
        'ROA (%)': 0.51,
    },
    {
        'Ano': 2017,
        'P/L': 8.58,
        'PSR': 0.62,
        'P/VP': 1.04,
        'DY (%)': 2.96,
        'MARGEM LÍQUIDA (%)': 7.23,
        'MARGEM EBIT (%)': 10.83,
        'ROE (%)': 10.92,
        'MARGEM BRUTA (%)': 21.31,
        'P/EBIT': 5.72,
        'ROA (%)': 0.79,
    },
    {
        'Ano': 2018,
        'P/L': 9.63,
        'PSR': 1.11,
        'P/VP': 1.45,
        'DY (%)': 3.26,
        'MARGEM LÍQUIDA (%)': 11.50,
        'MARGEM EBIT (%)': 16.98,
        'ROE (%)': 13.57,
        'MARGEM BRUTA (%)': 31.68,
        'P/EBIT': 6.53,
        'ROA (%)': 0.99,
    },
    {
        'Ano': 2019,
        'P/L': 9.23,
        'PSR': 1.23,
        'P/VP': 1.53,
        'DY (%)': 4.84,
        'MARGEM LÍQUIDA (%)': 13.32,
        'MARGEM EBIT (%)': 9.05,
        'ROE (%)': 15.22,
        'MARGEM BRUTA (%)': 27.50,
        'P/EBIT': 13.59,
        'ROA (%)': 1.07,
    },
    {
        'Ano': 2020,
        'P/L': 8.36,
        'PSR': 1.13,
        'P/VP': 0.95,
        'DY (%)': 3.81,
        'MARGEM LÍQUIDA (%)': 13.47,
        'MARGEM EBIT (%)': 11.73,
        'ROE (%)': 10.63,
        'MARGEM BRUTA (%)': 56.18,
        'P/EBIT': 9.60,
        'ROA (%)': 0.75,
    },
    {
        'Ano': 2021,
        'P/L': 4.19,
        'PSR': 0.66,
        'P/VP': 0.58,
        'DY (%)': 7.86,
        'MARGEM LÍQUIDA (%)': 15.66,
        'MARGEM EBIT (%)': 19.05,
        'ROE (%)': 13.82,
        'MARGEM BRUTA (%)': 47.38,
        'P/EBIT': 3.45,
        'ROA (%)': 0.99,
    },
    {
        'Ano': 2022,
        'P/L': 3.32,
        'PSR': 0.42,
        'P/VP': 0.62,
        'DY (%)': 12.01,
        'MARGEM LÍQUIDA (%)': 12.68,
        'MARGEM EBIT (%)': 16.26,
        'ROE (%)': 18.79,
        'MARGEM BRUTA (%)': 31.43,
        'P/EBIT': 2.59,
        'ROA (%)': 1.49,
    },
    {
        'Ano': 2023,
        'P/L': 4.79,
        'PSR': 0.60,
        'P/VP': 0.94,
        'DY (%)': 8.26,
        'MARGEM LÍQUIDA (%)': 12.49,
        'MARGEM EBIT (%)': 15.50,
        'ROE (%)': 19.60,
        'MARGEM BRUTA (%)': 33.43,
        'P/EBIT': 3.86,
        'ROA (%)': 1.54,
    },
    {
        'Ano': 2024,
        'P/L': 4.56,
        'PSR': 0.55,
        'P/VP': 0.82,
        'DY (%)': 10.35,
        'MARGEM LÍQUIDA (%)': 12.13,
        'MARGEM EBIT (%)': 12.73,
        'ROE (%)': 17.87,
        'MARGEM BRUTA (%)': 37.76,
        'P/EBIT': 4.35,
        'ROA (%)': 1.37,
    }
]

# Criar o DataFrame
df = pd.DataFrame(data_anualbb)

# Definir a ordem das colunas (opcional)
colunas = ['Ano', 'P/L', 'PSR', 'P/VP', 'DY (%)', 'MARGEM BRUTA (%)', 'MARGEM EBIT (%)', 'MARGEM LÍQUIDA (%)', 'P/EBIT', 'ROE (%)', 'ROA (%)']
df = df[colunas]



# Exibir o DataFrame
print(df)


     Ano    P/L   PSR  P/VP  DY (%)  MARGEM BRUTA (%)  MARGEM EBIT (%)  \
0   2011   5.35  0.73  1.17    7.93             32.48            19.21   
1   2012   6.52  0.78  1.20    6.33             32.05            15.94   
2   2013   6.70  0.67  1.00    9.68             28.88            12.30   
3   2014   5.10  0.49  0.90    6.97             23.13            11.33   
4   2015   2.67  0.23  0.61   11.42             12.32             5.56   
5   2016  11.45  0.46  1.06    2.65             17.58             6.18   
6   2017   8.58  0.62  1.04    2.96             21.31            10.83   
7   2018   9.63  1.11  1.45    3.26             31.68            16.98   
8   2019   9.23  1.23  1.53    4.84             27.50             9.05   
9   2020   8.36  1.13  0.95    3.81             56.18            11.73   
10  2021   4.19  0.66  0.58    7.86             47.38            19.05   
11  2022   3.32  0.42  0.62   12.01             31.43            16.26   
12  2023   4.79  0.60  0.94    8.26   

In [9]:
url = "https://investidor10.com.br/acoes/bbas3/"
response = requests.get(url)
response

<Response [403]>